<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Content-Based Personalization with LightGBM on Spark

This notebook provides a quick example of how to train a [LightGBM](https://github.com/Microsoft/Lightgbm) model on Spark using [MMLSpark](https://github.com/Azure/mmlspark) for a content-based personalization scenario.

We use the [CRITEO dataset](https://www.kaggle.com/c/criteo-display-ad-challenge), a well known dataset of website ads that can be used to optimize the Click-Through Rate (CTR). The dataset consists of a series of numerical and categorical features and a binary label indicating whether the add has been clicked or not.

The model is based on [LightGBM](https://github.com/Microsoft/Lightgbm), which is a gradient boosting framework that uses tree-based learning algorithms. Finally, we take advantage of
[MMLSpark](https://github.com/Azure/mmlspark) library, which allows LightGBM to be called in a Spark environment and be computed distributely.

This scenario is a good example of **implicit feedback**, where binary labels indicate the interaction between a user and an item. This contrasts with explicit feedback, where the user explicitely rate the content, for example from 1 to 5. 


## Global Settings and Imports

This notebook can be run in a Spark environment in a DSVM or in Azure Databricks. For more details about the installation process, please refer to the [setup instructions](../../SETUP.md).

**NOTE for Azure Databricks:**
* A python script is provided to simplify setting up Azure Databricks with the correct dependencies. Run ```python tools/databricks_install.py -h``` for more details.
* MMLSpark should not be run on a cluster with autoscaling enabled. Disable the flag in the Azure Databricks Cluster configuration before running this notebook.

In [1]:
import os
import sys

In [2]:
import findspark
spark_location='C:/spark-2.4.8-bin-hadoop2.7' # Set your own
java_location= 'C:/Program Files/Java/jdk1.8.0_291' # Set your own
os.environ['JAVA_HOME'] = java_location
findspark.init(spark_home=spark_location) 

In [3]:
import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm
import scrapbook as sb

In [4]:
from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

In [5]:
%env JAVA_HOME=C:/Program Files/Java/jdk1.8.0_291
%env HADOOP_HOME=C:/spark-2.4.8-bin-hadoop2.7/bin
%env SPARK_HOME=C:/spark-2.4.8-bin-hadoop2.7

env: JAVA_HOME=C:/Program Files/Java/jdk1.8.0_291
env: HADOOP_HOME=C:/spark-2.4.8-bin-hadoop2.7/bin
env: SPARK_HOME=C:/spark-2.4.8-bin-hadoop2.7


In [6]:
%env PYSPARK_DRIVER_PYTHON=jupyter

env: PYSPARK_DRIVER_PYTHON=jupyter


In [7]:
%env PYSPARK_DRIVER_PYTHON_OPTS=notebook

env: PYSPARK_DRIVER_PYTHON_OPTS=notebook


In [8]:
# Setup MML Spark
if not is_databricks():
    # get the maven coordinates for MML Spark from databricks_install script
    from tools.databricks_install import MMLSPARK_INFO
    print(1)
    packages = [MMLSPARK_INFO["maven"]["coordinates"]]
    print(2)
    repo = MMLSPARK_INFO["maven"].get("repo")
    print(3)
    spark = start_or_get_spark(packages=packages, repository=repo)
    print(4)
    dbutils = None
    print("MMLSpark version: {}".format(MMLSPARK_INFO['maven']['coordinates']))

1
2
3
4
MMLSpark version: com.microsoft.ml.spark:mmlspark_2.11:0.18.1


In [9]:
from mmlspark.train import ComputeModelStatistics
from mmlspark.lightgbm import LightGBMClassifier

In [10]:
print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

System version: 3.6.11 (default, Nov 27 2020, 18:37:51) [MSC v.1916 64 bit (AMD64)]
PySpark version: 2.4.8


In [11]:
# Criteo data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
# NUM_LEAVES = 32
NUM_LEAVES = 25
# NUM_ITERATIONS = 50
NUM_ITERATIONS = 10
LEARNING_RATE = 0.1
# FEATURE_FRACTION = 0.8
FEATURE_FRACTION = 0.2
# EARLY_STOPPING_ROUND = 10
EARLY_STOPPING_ROUND = 3

# Model name
MODEL_NAME = 'lightgbm_criteo.mml'

## Data Preparation

The [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) (Criteo DAC) dataset is a well-known industry benchmarking dataset for developing CTR prediction models, and is used frequently by research papers. The original dataset contains over 45M rows, but there is also a down-sampled dataset which has 100,000 rows (this can be used by setting `DATA_SIZE = "sample"`). Each row corresponds to a display ad served by Criteo and the first column is indicates whether this ad has been clicked or not.<br><br>
The dataset contains 1 label column and 39 feature columns, where 13 columns are integer values (int00-int12) and 26 columns are categorical features (cat00-cat25).<br><br>
What the columns represent is not provided, but for this case we can consider the integer and categorical values as features representing the user and / or item content. The label is binary and is an example of implicit feedback indicating a user's interaction with an item. With this dataset we can demonstrate how to build a model that predicts the probability of a user interacting with an item based on available user and item content features.


In [12]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)

100%|████████████████████████████████████████████████████████████████████████████| 8.58k/8.58k [00:02<00:00, 2.99kKB/s]


In [13]:
# visualize data
raw_data.limit(2).toPandas().head()

,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655


### Feature Processing
The feature data provided has many missing values across both integer and categorical feature fields. In addition the categorical features have many distinct values, so effectively cleaning and representing the feature data is an important step prior to training a model.<br><br>
One of the simplest ways of managing both features that have missing values as well as high cardinality is to use the hashing trick. The [FeatureHasher](http://spark.apache.org/docs/latest/ml-features.html#featurehasher) transformer will pass integer values through and will hash categorical features into a sparse vector of lower dimensionality, which can be used effectively by LightGBM.<br><br>
First, the dataset is splitted randomly for training and testing and feature processing is applied to each dataset.

In [14]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [15]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [16]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

## Model Training
In MMLSpark, the LightGBM implementation for binary classification is invoked using the `LightGBMClassifier` class and specifying the objective as `"binary"`. In this instance, the occurrence of positive labels is quite low, so setting the `isUnbalance` flag to true helps account for this imbalance.<br><br>

### Hyper-parameters
Below are some of the key [hyper-parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst) for training a LightGBM classifier on Spark:
- `numLeaves`: the number of leaves in each tree
- `numIterations`: the number of iterations to apply boosting
- `learningRate`: the learning rate for training across trees
- `featureFraction`: the fraction of features used for training a tree
- `earlyStoppingRound`: round at which early stopping can be applied to avoid overfitting

In [17]:
lgbm = LightGBMClassifier(
    labelCol="label",
    featuresCol="features",
    objective="binary",
    isUnbalance=True,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND
)

### Model Training and Evaluation

In [18]:
%load_ext ipycache

C:\Users\michen\anaconda3\envs\reco_pyspark\lib\site-packages\IPython\config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
C:\Users\michen\anaconda3\envs\reco_pyspark\lib\site-packages\ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [19]:
%%cache model.pkl model
model = lgbm.fit(train)

Py4JJavaError: An error occurred while calling o106.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 6.0 failed 1 times, most recent failure: Lost task 3.0 in stage 6.0 (TID 26, localhost, executor driver): java.lang.Exception: Dataset create call failed in LightGBM with error: bad allocation
	at com.microsoft.ml.spark.lightgbm.LightGBMUtils$.validate(LightGBMUtils.scala:31)
	at com.microsoft.ml.spark.lightgbm.LightGBMUtils$.generateSparseDataset(LightGBMUtils.scala:246)
	at com.microsoft.ml.spark.lightgbm.TrainUtils$.generateDataset(TrainUtils.scala:79)
	at com.microsoft.ml.spark.lightgbm.TrainUtils$.translate(TrainUtils.scala:258)
	at com.microsoft.ml.spark.lightgbm.TrainUtils$.trainLightGBM(TrainUtils.scala:411)
	at com.microsoft.ml.spark.lightgbm.LightGBMBase$$anonfun$6.apply(LightGBMBase.scala:85)
	at com.microsoft.ml.spark.lightgbm.LightGBMBase$$anonfun$6.apply(LightGBMBase.scala:85)
	at org.apache.spark.sql.execution.MapPartitions

ValueError: Variable(s) 'model' could not be found in the interactive namespace

In [ ]:
%%cache predictions.pkl predictions
predictions = model.transform(test)

In [ ]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

result = evaluator.transform(predictions)
auc = result.select("AUC").collect()[0][0]
result.show()

In [ ]:
# Record results with papermill for tests
sb.glue("auc", auc)

## Model Saving 
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [ ]:
%%cache pipeline.pkl pipeline
# save model
pipeline = PipelineModel(stages=[feature_processor, model])

In [ ]:
pipeline.write().overwrite().save(MODEL_NAME)

In [ ]:
# cleanup spark instance
if not is_databricks():
    spark.stop()

## Additional Reading
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154. https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf <br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>
